In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,990 kB]
Get:13 http://ppa.la

In [ ]:
#Start a Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [ ]:
#Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://amazonproductsreviewbucket.s3.us-west-2.amazonaws.com/amazon_reviews_us_Major_Appliances_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...|2015-08-31 00:00:00|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|M

In [ ]:
#Re-create vine_table from Deliverable 1
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

In [ ]:
#Seeing how many rows are in original vine_df data.
vine_df.count()

96901

In [ ]:
#Step 1. Filter data to retrieve all rows where total_votes count is equal to or greater than 20
filtered_vine_df = vine_df.filter(vine_df["total_votes"] >= 20)
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|     

In [ ]:
#Checking count on filtered_vine_df to compare to vine_df count.
filtered_vine_df.count()

5364

In [ ]:
#Step 2. Create new DataFrame using filtered DataFrame from Step 1 that retrieves all rows where the 
#number of helpful_votes/total_votes is >= 50%
second_filter_vine_df = filtered_vine_df.filter((filtered_vine_df["helpful_votes"])/(filtered_vine_df["total_votes"]) >= .50)
second_filter_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|     

In [ ]:
#Checking count on second_filter_vine_df to make sure items did get filtered out of filtered_vine_df
second_filter_vine_df.count()

4992

In [ ]:
#Step 3. Create a new DataFrame using DataFrame created in Step 2 that retrieves all rows where vine column is equal to 'Y'
paid_review_df = second_filter_vine_df.filter(second_filter_vine_df['vine'] == 'Y')
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20O97P5V9CZ21|          3|           21|         27|   Y|                N|
| RCNB7YGLWKCV5|          5|           23|         30|   Y|                N|
| RX1ZUVGBH3HKC|          5|          814|        848|   Y|                N|
| RRS5AGL10DYU3|          3|           25|         29|   Y|                N|
| R5YFC5G8V2B3I|          5|          104|        117|   Y|                N|
|R1IDQ0G1S7J9TN|          5|          178|        196|   Y|                N|
| RKXNZX22HQZEE|          3|          182|        192|   Y|                N|
|R2K45IY0NO70ZI|          5|          116|        122|   Y|                N|
| RQXRI4UKDX877|          5|           20|         24|   Y|                N|
|R1L2R8OHO171YG|          5|           66|         68|   Y|     

In [ ]:
#Step 4. Create a new DataFrame using DataFrame created in Step 2 that retrieves all rows where vine column is equal to 'N'
unpaid_review_df = second_filter_vine_df.filter(second_filter_vine_df['vine'] == 'N')
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|     

In [ ]:
#Step 5a. Determine total number of reviews
total_number_reviews = second_filter_vine_df.count()
total_number_reviews

4992

In [ ]:
#Step5b. Determine number of 5-star reviews
total_five_star_reviews = second_filter_vine_df.filter(second_filter_vine_df["star_rating"] == 5).count()
total_five_star_reviews

1981

In [ ]:
#Step5c. Determine percentage of 5-star reviews for paid reviews.
total_paid_reviews = paid_review_df.count()
total_paid_five_star_reviews = paid_review_df.filter(paid_review_df['star_rating'] == 5).count()
percentage_five_star_reviews_paid = round((total_paid_five_star_reviews/total_paid_reviews*100),2)
print(f'Out of the total number of Vine(paid) reviews, {percentage_five_star_reviews_paid}% were 5-star reviews.')

Out of the total number of Vine(paid) reviews, 51.43% were 5-star reviews.


In [ ]:
#Step5d. Determine percentage of 5-star reviews for unpaid reviews.
total_unpaid_reviews = unpaid_review_df.count()
total_unpaid_five_star_reviews = unpaid_review_df.filter(unpaid_review_df['star_rating'] == 5).count()
percentage_five_star_reviews_unpaid = round((total_unpaid_five_star_reviews/total_unpaid_reviews*100), 2)
print(f'Out of the total number of unpaid reviews, {percentage_five_star_reviews_unpaid}% were 5-star reviews')

Out of the total number of unpaid reviews, 39.6% were 5-star reviews


**Additional information to be used for Deliverable 3 (Analysis)**

In [ ]:
#Total number of Vine(paid) Reviews
total_paid_reviews

35

In [ ]:
#Total number of Non-Vine(unpaid) Reviews
total_unpaid_reviews

4957

In [ ]:
#Total number of Vine(paid) 5-star Reviews.
total_paid_five_star_reviews

18

In [ ]:
#Total number of Non-Vine(unpaid) Reviews
total_unpaid_five_star_reviews

1963